# Building Graphical Interfaces for ctcsound with PySimpleGUI

There are many GUI toolkits for Python. An overview can be found [here](https://wiki.python.org/moin/GuiProgramming). We will choose [PySimpleGui](http://www.pysimplegui.com/) here. For running ctcsound, we will use iCsound.

## Basic PySimpleGUI Concepts

Once PySimpleGUI is installed via pip or conda, it can be imported, by convention as *sg*.

In [ ]:
import PySimpleGUI as sg

The code for a GUI has two main parts:
- The **layout** containing the *widgets* (mostly called *elements*) in the **window**.
- The main **event loop** in which the widget's information are being read.

This is a simple example:

In [ ]:
layout = [[sg.Input(key='INPUT')],
          [sg.Button('Read'), sg.Exit()]]

window = sg.Window('Please type', layout)

while True:
    event, values = window.read()
    print(event, values)
    if event is None or event == 'Exit':
        break

window.close()

- The layout consists of lists which each contains the widgets for one line.
- The `sg.Input()` element is the one which gets the text input. So we give it a **key** as signifier. Usually the key is a string.
- In the event loop, we read the `values` once an event (here: a press on the *Read* button) has been received.
- As can be seen in the printout, *values* are a python dictionnary. If we want to retrieve the data, we will write `values['INPUT']`
- The `if` condition gives two options for closing the window: either by clicking the *Exit* button, or by closing with the X.

## GUI -> Csound

This knowledge is actually enough to start using a PySimpleGUI in iCsound. We will first look at examples where Csound parameters are controlled by widgets.

### Slider

First, we start iCsound and send some code to the running instance.

In [ ]:
%load_ext csoundmagics
cs = ICsound()
orc = """
instr 1
 kFreq chnget "freq"
 aOut poscil .2, kFreq
 out aOut, aOut
endin
schedule(1,0,-1)"""
cs.sendCode(orc)

Now we create a GUI in which we send the slider values to the channel "freq".

In [ ]:
layout = [[sg.Slider(range=(500,1000),orientation='h',key='FREQ',enable_events=True)]]

window = sg.Window('Simple Slider',layout)

while True:
    event, values = window.read()
    if event is None:
        break
    cs.setControlChannel('freq',values['FREQ'])
        
window.close()

And we have to turnoff our instrument and delete the iCsound instance.

In [ ]:
cs.sendScore('i -1 0 1')
del cs

### Button

A button can have different functions. We use here one button to browse a sound file, and one button to start/stop the performance.

At first we create th iCsound instance and send some new code to it.

In [ ]:
cs = ICsound()
orc = """
instr Playback
 Sfile chnget "file"
 aFile[] diskin Sfile,1,0,1
 kFadeOut linenr 1, 0, 1, .01
 out aFile*kFadeOut
endin
"""
cs.sendCode(orc)

Then we build a GUI with three buttons:
- A file browser. We add a string as key, and we set `enable_events=True` to send the browsed file as event.
- A *Start* button. This button will send the score `i "Playback" 0 -1` to activate instrument *Playback*. This instrument will play the sound file in a loop.
- A *Stop* button. This button will stop the instrument. We can stop an instrument with negative p3 by calling .it with negative p1. As we are using instrument names here, we must first transform the name "Playback" to the number Csound has given it: `-nstrnum("Playback")` does the trick.  

Once the window is being closed, we delete the Csound instance as usual with `del cs`.

In [ ]:
layout = [[sg.Text('Select Audio File, then Start/Stop Playback')],      
          [sg.FileBrowse(key='FILE',enable_events=True), sg.Button('Start'), sg.Button('Stop')]]

window = sg.Window('',layout)

while True:
    event, values = window.read()
    if event is None:
        break
    cs.setStringChannel('file',values['FILE'])
    if event is 'Start':
        cs.sendScore('i "Playback" 0 -1')
    if event is 'Stop':
        cs.sendCode('schedule(-nstrnum("Playback"),0,0)')

cs.stopEngine()
del cs
window.close()

### Spinbox, Menu, Checkbox, Radiobutton

to be added ...

## Csound -> GUI

We will now look at the opposite direction: Sending signals from Csound to the GUI.

### Basic Example

To see how things are working here, we only create a line in Csound which randomly moves between -1 and 1. We send this k-signal in a channel called "line". 

On the PySimpleGUI side there is one important change to the approach so far. As we have no events which are triggered by any user action, we must must include a **timeout** statement in the `window.read()` call. Thie will update the widgets every *timeout* milliseconds. So *timeout=100* will refresh the GUI ten times per second.

Reading the Csound channel in the Python environment is done via iCsound's `channel` method. The return value of `cs.channel('line')`, however, is not the channel value itself, but a tuple of the value and the data type. So to retrieve the value we have to extract the first element by `cs.channel('line')[0]`.

In [ ]:
# csound start
cs = ICsound()
orc = """
instr 1
 kLine randomi -1,1,1,3
 chnset kLine, "line"
endin
"""
cs.sendCode(orc)
cs.sendScore('i 1 0 -1')

# GUI
layout = [[sg.Slider(range=(-1,1),
                     orientation='h',
                     key='LINE',
                     disable_number_display=True,
                     resolution=.01)],
          [sg.Text(size=(6,1),
                   key='LINET',
                   text_color='black',
                   background_color='white',
                   justification = 'right',
                   font=('Courier',16,'bold'))]
         ]

# window and event loop
window = sg.Window('Simple Csound -> GUI Example',layout)
while True:
    event, values = window.read(timeout=100)
    if event is None:
        cs.sendScore('i -1 0 1')
        del cs
        break
    window['LINE'].update(cs.channel('line')[0])
    window['LINET'].update('%+.3f' % cs.channel('line')[0])
window.close()

### Display Csound Tables

The *Graph* element in PySimpleGUI can be used to display Csound function tables. Currently (June 2020) the Graph element is missing a method to plot an array straightforward. So at first we define a function for it.

In [ ]:
def plotArray(graph, y=[], linewidth=2, color='black'):
    """Plots the array y in the graph element.
    Assumes that x of  graph is 0 to tablelen."""
    import numpy as np
    for i in np.arange(len(y)-1):
        graph.draw_line((i,y[i]), (i+1,y[i+1]), color=color, width=linewidth)

For static display of a table we will put the function outside the event loop. We get the table data with iCsound's *table* method.

In [ ]:
cs = ICsound()

orc = """
i0 ftgen 1, 0, 0, 1, "examples/fox.wav", 0, 0, 0
"""
cs.sendCode(orc)
tablen = cs.tableLength(1)

layout = [[sg.Graph(canvas_size=(400, 200), 
                    graph_bottom_left=(0, -1), 
                    graph_top_right=(tablen, 1),
                    background_color='white',
                    key='graph')]]

window = sg.Window('csound table', layout, finalize=True)
table_disp = window['graph']

y = cs.table(1)
plotArray(table_disp, y)

while True:
    event, values = window.read()
    if event is None:
        cs.stopEngine()
        del cs
        break
        
window.close()

If we want to show dynamically changing tables, we will put the plot function in the event loop. Table data and display are updated then every *timeout* milliseconds. In the next example an additive synthesis instrument plays some sine partials. After three seconds the table is being blurred slowly.

In [ ]:
cs = ICsound()
orc = """
seed 0
i0 ftgen 1, 0, 1024, 10, 1
instr Blur
 indx = 0
 while indx<1024 do
   tablew limit:i(table:i(indx,1)+rnd31:i(1/p4,0),-1,1), indx, 1
   indx += 1
  od
 schedule("Blur",.1,0,limit:i(p4-p4/100,1,p4))
endin
instr Play
 iFreq random 500, 510
 a1 poscil transeg:a(1/10,p3,-3,0), iFreq, 1
 a2 poscil transeg:a(1/20,p3,-5,0), iFreq*2.32, 1
 a3 poscil transeg:a(1/30,p3,-6,0), iFreq*4.25, 1
 a4 poscil transeg:a(1/40,p3,-7,0), iFreq*6.63, 1
 a5 poscil transeg:a(1/50,p3,-8,0), iFreq*9.38, 1
 aSnd sum a1, a2, a3, a4, a5
 aSnd butlp aSnd, 5000
 aL, aR pan2 aSnd, random:i(0,1)
 out aL, aR
 schedule("Play", random:i(.3,1.5), random:i(2,4))
endin
schedule("Play",0,3)
schedule("Blur",3,0,100)
"""
cs.sendCode(orc)
tablen = cs.tableLength(1)

layout = [[sg.Graph(canvas_size=(400, 200), 
                    graph_bottom_left=(0, -1), 
                    graph_top_right=(tablen, 1),
                    background_color='white',
                    key='graph')]]

window = sg.Window('csound table real time modifications', layout, finalize=True)
table_disp = window['graph']

while True:
    event, values = window.read(timeout=100)
    if event is None:
        cs.stopEngine()
        del cs
        break
    y = cs.table(1)
    table_disp.erase()
    plotArray(table_disp, y)

window.close()

Written by Joachim Heintz, June 2020.